In [36]:
!pip install requests
!pip install transformers

In [37]:
import requests

url = 'https://gutenberg.org/cache/epub/35688/pg35688.txt'  # Replace <BOOK_ID> with the actual book ID from Gutenberg
response = requests.get(url)

file_name = "wonderland.txt"
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(response.text)

print("Book downloaded and saved as", file_name)

with open(file_name, 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:500])

Book downloaded and saved as wonderland.txt
﻿The Project Gutenberg eBook of Alice in Wonderland
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.



In [38]:
print(text[10022:11066])


RED QUEEN

It's time for you to answer now; open your mouth a _little_ wider when you
speak, and always say, "Your Majesty." I suppose you don't want to lose
your name?


ALICE

No, indeed.


RED QUEEN

And yet I don't know, only think how convenient it would be if you could
manage to go home without it! For instance, if the governess wanted to
call you to your lessons, she would call out "come here," and there she
would have to leave off, because there wouldn't be any name for her to
call, and of course you wouldn't have to go, you know.


ALICE

That would never do, I'm sure; the governess would never think of excusing
me from lessons for that. If she couldn't remember my name, she'd call me
"Miss," as the servants do.


RED QUEEN

Well, if she said "Miss," and didn't say anything more, of course you'd
miss your lessons. I dare say you can't even read this book.


ALICE

It's all in some language I don't know. Why, it's a looking-glass book, of
course! And if I hold it up to a glass

In [39]:
from transformers import pipeline

summarizer = pipeline("summarization")

part_to_summarize = text[10022:11066]

summarized = summarizer(part_to_summarize)[0]['summary_text']

print(summarized)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Red Queen: "Open your mouth a little wider when you speak, and always say, "Your Majesty" I suppose you don't want to lose your name? I don't know, only think how convenient it would be if you could go home without it! For instance, if the governess wanted to call you to your lessons, she would call out "come here," and there she would have to leave off because there wouldn't be any name for her to call, and of course you wouldn't have to go .


In [ ]:
from summarizer import summarize
import torch
from transformers import BertTokenizer
model = 'model.pth'
model = torch.load(model)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
summary = summarize(model, tokenizer, part_to_summarize)

In [ ]:
! pip install rouge-score

In [41]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(part_to_summarize, summarized)

print(f"ROUGE Scores: {scores}")

ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.4225352112676056, fmeasure=0.5940594059405941), 'rouge2': Score(precision=0.9662921348314607, recall=0.4056603773584906, fmeasure=0.5714285714285715), 'rougeL': Score(precision=1.0, recall=0.4225352112676056, fmeasure=0.5940594059405941)}


In [42]:
import re
from collections import Counter

# Function to preprocess the text
def preprocess_text(text):
    sentences = text.split('.')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    words = [re.findall(r'\w+', sentence.lower()) for sentence in sentences]
    return sentences, words

# Function to calculate word frequencies
def calculate_word_frequencies(words):
    all_words = [word for sentence in words for word in sentence]
    stopwords = set(["the", "is", "in", "and", "to", "a", "of", "it", "on", "for", "with", "as", "was", "at"])  # Add more stopwords as needed
    filtered_words = [word for word in all_words if word not in stopwords]
    return Counter(filtered_words)

# Function to score sentences
def score_sentences(sentences, words, word_freq):
    sentence_scores = {}
    for i, sentence in enumerate(words):
        sentence_score = sum(word_freq[word] for word in sentence if word in word_freq)
        sentence_scores[i] = sentence_score
    return sentence_scores

# Function to generate a summary
def generate_summary(sentences, sentence_scores, n=3):
    # Get top N sentences based on scores
    ranked_sentences = sorted(sentence_scores.keys(), key=lambda x: sentence_scores[x], reverse=True)[:n]
    summary = [sentences[i] for i in sorted(ranked_sentences)]
    return ' '.join(summary)

# Preprocess the text
sentences, words = preprocess_text(part_to_summarize)

# Calculate word frequencies
word_freq = calculate_word_frequencies(words)

# Score sentences
sentence_scores = score_sentences(sentences, words, word_freq)

# Generate summary
summary = generate_summary(sentences, sentence_scores, n=2)

print("Summary:")
print(summary)

Summary:
RED QUEEN

And yet I don't know, only think how convenient it would be if you could
manage to go home without it! For instance, if the governess wanted to
call you to your lessons, she would call out "come here," and there she
would have to leave off, because there wouldn't be any name for her to
call, and of course you wouldn't have to go, you know RED QUEEN

Well, if she said "Miss," and didn't say anything more, of course you'd
miss your lessons


In [44]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(part_to_summarize, summary)

print(f"ROUGE Scores: {scores}")

ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.431924882629108, fmeasure=0.6032786885245902), 'rouge2': Score(precision=0.989010989010989, recall=0.42452830188679247, fmeasure=0.594059405940594), 'rougeL': Score(precision=1.0, recall=0.431924882629108, fmeasure=0.6032786885245902)}
